In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/moviesfinal.csv")

In [3]:
df.columns

Index(['Unnamed: 0', 'movie_id', 'Movie_name', 'tagline', 'countries',
       'release_date', 'director_name', 'homepage', 'genre_names', 'GenreID_s',
       'Actors', 'Actress', 'Other_cast', 'runtime', 'status', 'vote_average',
       'vote_count', 'directorID'],
      dtype='object')

In [4]:
import sqlite3
conn = sqlite3.connect('MovieFinder.db')
c= conn.cursor()


In [5]:
c.execute("""CREATE TABLE IF NOT EXISTS Movie(mid integer PRIMARY KEY,
                name text,
                description text,
                region text,
                released_date text,
                poster BLOB,
                director integer,
                rating integer,
                votecount integer,
                FOREIGN KEY (director) REFERENCES Person(pid))""")
c.execute("""CREATE TABLE IF NOT EXISTS Movie_Genre(movie integer,
                genretype text,
                genreid integer,
                FOREIGN KEY (movie) REFERENCES Movie(mid))""")

c.execute("""CREATE TABLE IF NOT EXISTS Person(pid integer PRIMARY KEY ,
                name text)""")




c.execute("""CREATE TABLE IF NOT EXISTS User(
                uid text PRIMARY KEY,
                name text,
                password text,
                profile_photo BLOB
                )""")
c.execute("""CREATE TABLE IF NOT EXISTS watch_history(
                uID text,
                mID integer,
                watch_date text,
                FOREIGN KEY (uID) REFERENCES User (uid),
                FOREIGN KEY (mID) REFERENCES Movie (mid)
                )""")

c.execute("""CREATE TABLE IF NOT EXISTS wish_list(
                userID text,
                movieID integer,
                FOREIGN KEY (userID) REFERENCES User(uid),
                FOREIGN KEY (movieID) REFERENCES Movie(mid)
                )""")

c.execute("""CREATE TABLE IF NOT EXISTS user_banned_list(
                userID text,
                banned_user_ID integer,
                FOREIGN KEY (userID) REFERENCES User(uid)
                )""")
c.execute("""CREATE TABLE IF NOT EXISTS Movie_Posters(
                movieID integer,
                poster BLOB,
                FOREIGN KEY (movieID) REFERENCES Movie(mid)
                )""")



In [6]:
c.execute("""CREATE TABLE IF NOT EXISTS Review(
                user text,
                movie integer, 
                comment text,
                rating integer,
                date text,
                FOREIGN KEY (movie) REFERENCES Movie(mid),
                FOREIGN KEY (user) REFERENCES User(uid)
                )""")

In [7]:

def insertintoMovieTable(mid,name,description,region,released_date,poster,director,rating,votecount):
    sqliteConnection = sqlite3.connect('MovieFinder.db')
    cursor = sqliteConnection.cursor()
    sqlite_insert_with_param = """REPLACE INTO Movie
        (mid,name,description,region,released_date,poster,director,votecount,rating) 
        VALUES (?, ?, ?, ?, ?,?,?,?,?);""" 
    data_tuple=(mid,name,description,region,released_date,poster,director,rating,votecount)
    cursor.execute(sqlite_insert_with_param, data_tuple)       
    sqliteConnection.commit()      
    cursor.close()
    

In [8]:
movies= pd.read_csv("data/moviesfinal.csv")
movies = movies[:200]
movies.columns

Index(['Unnamed: 0', 'movie_id', 'Movie_name', 'tagline', 'countries',
       'release_date', 'director_name', 'homepage', 'genre_names', 'GenreID_s',
       'Actors', 'Actress', 'Other_cast', 'runtime', 'status', 'vote_average',
       'vote_count', 'directorID'],
      dtype='object')

In [9]:
counter = 1
for index,row in movies.iterrows():
    counter +=1
    insertintoMovieTable(mid=row['movie_id'],name=row['Movie_name'],description=row['tagline'],region= row['countries'],released_date=row['release_date'],poster="posters/19995_avatar.jpg",director= row['directorID'],rating=row['vote_average'],votecount=row['vote_count'])
    

In [10]:

sqliteConnection = sqlite3.connect('MovieFinder.db')
cursor = sqliteConnection.cursor()
print("Connected to SQLite")
sqlite_select_query = """SELECT * from Movie"""
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
print(len(records))

Connected to SQLite
200


In [11]:
#c.execute("""CREATE TABLE IF NOT EXISTS Movie_Genre(mid,movie,genretype, genreid))
def insertintoMovie_GenreTable(movie,genretype, genreid):
    sqliteConnection = sqlite3.connect('MovieFinder.db')
    cursor = sqliteConnection.cursor()
    sqlite_insert_with_param = """REPLACE INTO Movie_Genre
          (movie,genretype,genreid)
          VALUES (?, ?, ?);""" 
    data_tuple=(movie,genretype, genreid)
    cursor.execute(sqlite_insert_with_param, data_tuple)       
    sqliteConnection.commit()      
    cursor.close()
    

In [12]:
df1 = pd.read_csv("data/Genre_table.csv")

In [13]:
for index,row in df1.iterrows():
    insertintoMovie_GenreTable(movie=row['movieID'],genretype=row['GenreName'], genreid=row['genreID'])

In [14]:
sqliteConnection = sqlite3.connect('MovieFinder.db')
cursor = sqliteConnection.cursor()
print("Connected to SQLite")
sqlite_select_query = """SELECT * from Movie_Genre"""
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
print(len(records))

Connected to SQLite
667


In [15]:
c.execute("""CREATE TABLE IF NOT EXISTS Person(pid integer,
                name text)""")
def insertintoPersonTable(pid,name):
    sqliteConnection = sqlite3.connect('MovieFinder.db')
    cursor = sqliteConnection.cursor()
    sqlite_insert_with_param = """REPLACE INTO Person
          (pid,name)
          VALUES (?, ?);""" 
    data_tuple=(pid,name)
    cursor.execute(sqlite_insert_with_param, data_tuple)       
    sqliteConnection.commit()      
    cursor.close()

In [16]:
df2= pd.read_csv("data/director.csv")
#df2.to_csv("director.csv",header=['ID','Name'],index=False)

In [17]:
for index,row in df2.iterrows():
    insertintoPersonTable(pid=row['ID'],name=row['Name'])

In [18]:
df3= pd.read_csv("data/cast_1.csv")

In [19]:
for index,row in df3.iterrows():
    insertintoPersonTable(pid=row['CastID'],name=row['castname'])

In [ ]:
sqliteConnection = sqlite3.connect('MovieFinder.db')
cursor = sqliteConnection.cursor()
print("Connected to SQLite")
sqlite_select_query = """SELECT * from Person"""
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
print(len(records))

In [ ]:
def convertToBinaryData(filename):
    with open(filename, 'rb') as file:
        blobData = file.read()
    return blobData
def insertintoUserTable(uid,name,password,profile_photo):
    sqliteConnection = sqlite3.connect('MovieFinder.db')
    cursor = sqliteConnection.cursor()
    sqlite_insert_with_param = """REPLACE INTO User
        (uid,name,password,profile_photo) 
        VALUES (?, ?, ?, ?);""" 
    Photo = convertToBinaryData(profile_photo)
    data_tuple = (uid,name,password,Photo)
    cursor.execute(sqlite_insert_with_param, data_tuple)       
    sqliteConnection.commit()      
    cursor.close()

In [ ]:
insertintoUserTable('1@gmail.com','Sri','hjgdssrsj9076',"movies/profiles/1")
insertintoUserTable('2@gmail.com','Pattrick','joigjkiophgp544',"movies/profiles/2.png")
insertintoUserTable('3@gmail.com','Zijian','dewygoqwihf3',"movies/profiles/3")
insertintoUserTable('4@gmail.com','Mingkai','dfhjfksuf',"movies/profiles/4.jpg")

In [ ]:

def insertintowatch_history(uID , mID ,watch_date):
    sqliteConnection = sqlite3.connect('MovieFinder.db')
    cursor = sqliteConnection.cursor()
    sqlite_insert_with_param = """REPLACE INTO watch_history
        (uID , mID ,watch_date) 
        VALUES (?, ?,?);""" 
    data_tuple = (uID , mID ,watch_date)
    cursor.execute(sqlite_insert_with_param, data_tuple)       
    sqliteConnection.commit()      
    cursor.close()



In [ ]:


def insertintowish_list(userID , movieID):
    sqliteConnection = sqlite3.connect('MovieFinder.db')
    cursor = sqliteConnection.cursor()
    sqlite_insert_with_param = """REPLACE INTO wish_list
        (userID , movieID) 
        VALUES (?, ?);""" 
    data_tuple = (userID , movieID )
    cursor.execute(sqlite_insert_with_param, data_tuple)       
    sqliteConnection.commit()      
    cursor.close() 

In [ ]:

def insertintouser_banned_list( userID,banned_user_ID):
    sqliteConnection = sqlite3.connect('MovieFinder.db')
    cursor = sqliteConnection.cursor()
    sqlite_insert_with_param = """REPLACE INTO user_banned_list
        ( userID,banned_user_ID)
        VALUES (?, ?);""" 
    data_tuple = (userID,banned_user_ID)
    cursor.execute(sqlite_insert_with_param, data_tuple)       
    sqliteConnection.commit()      
    cursor.close()     

In [ ]:

def insertintoMovie_PostersTable(movieID,poster):
    sqliteConnection = sqlite3.connect('MovieFinder.db')
    cursor = sqliteConnection.cursor()
    sqlite_insert_with_param = """REPLACE INTO Movie_Posters
        (movieID,poster) 
        VALUES (?, ?);""" 
    Photo = convertToBinaryData(poster)
    data_tuple = (movieID,Photo)
    cursor.execute(sqlite_insert_with_param, data_tuple)       
    sqliteConnection.commit()      
    cursor.close()

In [ ]:
#insertintoMovie_PostersTable(19995,"posters/19995_avatar.jpg")

In [ ]:
df6= pd.read_csv("data/Movie_Director.csv")
counter=0
for index,row in df6.iterrows():
    #counter +=1
    #print(counter,row['movie_id'])
    x=row['movie_id']
    insertintoMovie_PostersTable(movieID=x,poster= "movies/posters/"+str(x)+".jpg")

In [ ]:
sqliteConnection = sqlite3.connect('MovieFinder.db')
cursor = sqliteConnection.cursor()
print("Connected to SQLite")
sqlite_select_query = """SELECT * from Movie_Posters"""
cursor.execute(sqlite_select_query)
records = cursor.fetchall()
print(len(records))